In [57]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, Dataset, random_split
import numpy as np
from os.path import join
import mat73
import matplotlib.pyplot as plt
import optuna

In [58]:
# Load data into dictionary
DataPath = join("neuro_data","dataSubj10.mat")
data_dict = mat73.loadmat(DataPath, use_attrdict=True)

RAWDATA = data_dict["data"]
BATCH_SIZE=16
## CROPPING for data, 2.6s-5.6s, region of interest with audio
FS = 512
LO = int(2.6*FS) #1331
HI = int(5.5*FS) #2816
# Dif : 1485 = 3 * 3 * 3 * 5 * 11

In [59]:
class CreateDataset(Dataset):
    """Creates dataset for EEGNET, meaning move all relevant channel data into one matrix for x, and results into y

    Args:
        Dataset (_type_): Takes in data loaded from Matlab and formats appropriately.
    """
    
    def __init__(self, data, channels, crop=None):
        # Associates channel names with channel data in a dictionary
        datadicts = [dict(zip(np.squeeze(data["label"]),dat)) for dat in data["trial"]]

        x, y = [0]*len(data["trialinfo"]), np.array([0]*len(data["trialinfo"]))
        
        # Extract the y-values, i.e. which side the audio was played
        for i, trialinfo in enumerate(data["trialinfo"]):
            # side : left = 1, right = 0
            y[i] = int(trialinfo[0]["side"])==1
        
        self.y = y
        
        # Extract only information from relevant channels
        for i, dat in enumerate(datadicts):
            x[i] = [dat[ch] for ch in channels]
        x = np.array(x)
        
        # Include only specific section of time-series.
        if crop:
            x=x[:,:,crop[0]:crop[1]]
        
        self.x = x

    def __getitem__(self, index):
        feature = torch.tensor([self.x[index]], dtype=torch.float32)
        label = torch.tensor([self.y[index]], dtype=torch.float32)

        return feature, label
    
    def __len__(self):
        return len(self.x)
        

In [60]:
# https://towardsdatascience.com/convolutional-neural-networks-for-eeg-brain-computer-interfaces-9ee9f3dd2b81
class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class EEGNET(nn.Module):
    ''' Dimensions of key layers.
    in:
    [BATCH_SIZE,1,N_CHANNELS,HI-LO]
    temporal:
    [BATCH_SIZE,filter_sizing,N_CHANNELS,HI-LO]
    spatial:
    [BATCH_SIZE,filter_sizing*D,1,HI-LO]
    avgpool1:
    [BATCH_SIZE,filter_sizing*D,1,(HI-LO-mean_pool)/mean_pool + 1]
    avgpool2:
    [BATCH_SIZE,filter_sizing*D,1,floor((floor((HI-LO-mean_pool)/mean_pool + 1)-mean_pool)/mean_pool + 1)]
    '''
    def __init__(self, filter_sizing, dropout, D, channel_amount, receptive_field=512, mean_pool=15):
        super(EEGNET,self).__init__()
        self.temporal=nn.Sequential(
            nn.Conv2d(1,filter_sizing,kernel_size=[1,receptive_field],stride=1, bias=False,\
                padding='same'), 
            nn.BatchNorm2d(filter_sizing),
        )
        self.spatial=nn.Sequential(
            nn.Conv2d(filter_sizing,filter_sizing*D,kernel_size=[channel_amount,1],bias=False,\
                groups=filter_sizing),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ELU(True),
        )

        self.seperable=nn.Sequential(
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,16],\
                padding='same',groups=filter_sizing*D, bias=False),
            nn.Conv2d(filter_sizing*D,filter_sizing*D,kernel_size=[1,1], padding='same',groups=1, bias=False),
            nn.BatchNorm2d(filter_sizing*D),
            nn.ELU(True),
        )

        self.avgpool1 = nn.AvgPool2d([1, mean_pool], stride=[1, mean_pool], padding=0)   
        self.avgpool2 = nn.AvgPool2d([1, mean_pool], stride=[1, mean_pool], padding=0)
        self.dropout = nn.Dropout(dropout)
        self.view = nn.Sequential(Flatten())

        # Endsize calculated from dimensions given in documentation for Conv2d, AvgPool2d and Flatten.
        endsize = filter_sizing*D*np.floor((np.floor((HI-LO-mean_pool)/mean_pool + 1)-mean_pool)/mean_pool + 1)
        self.fc2 = nn.Linear(int(endsize), 1)

    def forward(self,x):
        out = self.temporal(x)
        out = self.spatial(out)
        out = self.avgpool1(out)
        out = self.dropout(out)
        out = self.seperable(out)
        out = self.avgpool2(out)
        out = self.dropout(out)
        out = out.view(out.size(0), -1)
        prediction = self.fc2(out)
        return torch.sigmoid(prediction)

In [61]:
def evaluate_loss(model, criterion, dataloader):
    model.eval()
    total_loss = 0.0
    for batch_X, batch_y in dataloader:
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        total_loss += loss.item()
        
    return total_loss / len(dataloader)

In [62]:
def evaluate_acc(model, dataloader):
    model.eval()
    total_acc = 0.0
    for batch_X, batch_y in dataloader:
        outputs = model(batch_X)
        predictions = 1.0*(outputs>0.5)
        total_acc += (predictions==batch_y).sum()
        
    return total_acc / len(dataloader.dataset)

In [63]:
def train(model, criterion, optimizer, train_loader, valid_loader, n_epochs):
    train_loss_list = []
    valid_loss_list = []
    train_acc_list = []
    valid_acc_list = []
    for epoch in range(1, n_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            ypred = model.forward(batch_X)
            loss = criterion(ypred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        train_loss = evaluate_loss(model, criterion, train_loader)
        valid_loss = evaluate_loss(model, criterion, valid_loader)
        train_acc = evaluate_acc(model, train_loader)
        valid_acc = evaluate_acc(model, valid_loader)
        train_loss_list.append(train_loss)
        valid_loss_list.append(valid_loss)
        train_acc_list.append(train_acc)
        valid_acc_list.append(valid_acc)

        print(f"| epoch {epoch:2d} | train loss {train_loss:.6f} | train acc {train_acc:.6f} | valid loss {valid_loss:.6f} | valid acc {valid_acc:.6f} |")

    return train_loss_list, valid_loss_list, train_acc_list, valid_acc_list

# Training

In [75]:
# fix random seed
np.random.seed(293210931)
torch.manual_seed(293210931)

channels = ["T7","FT7","TP7","TP8","FT8","T8"]
N_CHANNELS = len(channels)

DATASET = CreateDataset(RAWDATA, channels, [LO, HI])


dat_train, dat_val, dat_test = random_split(DATASET, [0.7,0.1,0.2])

train_loader = DataLoader(dat_train, batch_size=BATCH_SIZE)
val_loader = DataLoader(dat_val, batch_size=BATCH_SIZE)
test_loader = DataLoader(dat_test, batch_size=BATCH_SIZE)

# Set up elements
model = EEGNET(4,0.33,2,6,mean_pool=15)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 8.5e-5)

# Train network
train(model, criterion, optimizer, train_loader, val_loader, 100) #Should be val instead of test



| epoch  1 | train loss 0.695605 | train acc 0.489879 | valid loss 0.693758 | valid acc 0.371429 |
| epoch  2 | train loss 0.697846 | train acc 0.497976 | valid loss 0.693400 | valid acc 0.400000 |
| epoch  3 | train loss 0.698251 | train acc 0.510121 | valid loss 0.692138 | valid acc 0.457143 |
| epoch  4 | train loss 0.698119 | train acc 0.510121 | valid loss 0.690376 | valid acc 0.457143 |
| epoch  5 | train loss 0.696885 | train acc 0.522267 | valid loss 0.686340 | valid acc 0.514286 |
| epoch  6 | train loss 0.696198 | train acc 0.534413 | valid loss 0.684502 | valid acc 0.514286 |
| epoch  7 | train loss 0.695441 | train acc 0.538462 | valid loss 0.683022 | valid acc 0.514286 |
| epoch  8 | train loss 0.694486 | train acc 0.538462 | valid loss 0.680207 | valid acc 0.600000 |
| epoch  9 | train loss 0.693803 | train acc 0.538462 | valid loss 0.679584 | valid acc 0.600000 |
| epoch 10 | train loss 0.692887 | train acc 0.534413 | valid loss 0.677372 | valid acc 0.571429 |
| epoch 11

([0.6956054233014584,
  0.6978459879755974,
  0.6982506513595581,
  0.6981189250946045,
  0.6968852803111076,
  0.6961981728672981,
  0.6954405345022678,
  0.6944855153560638,
  0.6938026584684849,
  0.6928866654634476,
  0.6924872063100338,
  0.6917837746441364,
  0.6910897195339203,
  0.6908866539597511,
  0.6904580034315586,
  0.689185380935669,
  0.6887613087892532,
  0.688357450067997,
  0.6878504864871502,
  0.6872808374464512,
  0.6868154592812061,
  0.6869816780090332,
  0.6864540688693523,
  0.6859642937779427,
  0.6856733560562134,
  0.685358788818121,
  0.6850273944437504,
  0.684711541980505,
  0.6844332069158554,
  0.6843451969325542,
  0.684350561350584,
  0.6839914470911026,
  0.6836360208690166,
  0.6832968853414059,
  0.6830139048397541,
  0.6826390437781811,
  0.6822922304272652,
  0.6819493323564529,
  0.6818790771067142,
  0.6816242001950741,
  0.6814632713794708,
  0.6810877248644829,
  0.6807973347604275,
  0.6807082965970039,
  0.6804441623389721,
  0.68024013191

# Hyperparam optimization Optuna

Here I attempt to optimize hyper parameter choice to maximize accuracy on validation data.

In [65]:
def build_model(params):
    fs = params["filter_sizing"]
    do = params["dropout"]
    D = params["D"]
    rf = params["receptive_field"]
    return EEGNET(fs, do, D, N_CHANNELS, receptive_field=rf, mean_pool=15)

def train_and_eval(params, model, n_epochs=30):

    # Load in the data
    dat_train, dat_val, dat_test = random_split(DATASET, [0.7,0.1,0.2])

    # FIXME: Do random split outside function to secure untouched test data, then split the remaining inside train_and_eval
    train_loader = DataLoader(dat_train, batch_size=BATCH_SIZE)
    valid_loader = DataLoader(dat_val, batch_size=BATCH_SIZE)
    test_loader = DataLoader(dat_test, batch_size=BATCH_SIZE)
    
    # Set Loss criterion and extract optimizer
    criterion = nn.BCELoss()
    optimizer_options = params["optimizer"]
    optimizer = getattr(torch.optim, optimizer_options)(model.parameters(), lr = params["learning_rate"])

    for epoch in range(1, n_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            ypred = model.forward(batch_X)
            loss = criterion(ypred, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        train_loss = evaluate_loss(model, criterion, train_loader)
        valid_loss = evaluate_loss(model, criterion, valid_loader)
        train_acc = evaluate_acc(model, train_loader)
        valid_acc = evaluate_acc(model, valid_loader)

        print(f"| epoch {epoch:2d} | train loss {train_loss:.6f} | train acc {train_acc:.6f} | valid loss {valid_loss:.6f} | valid acc {valid_acc:.6f} |")

    return valid_acc

def objective(trial):

    params = {
        "filter_sizing" : trial.suggest_int("filter_sizing", 1, 8), 
        "dropout" : trial.suggest_uniform("dropout", 0.01, 1), 
        "D" : trial.suggest_int("Depth Parameter", 1, 4), 
        "receptive_field" : trial.suggest_int("receptive_field", 64, 512, step=64),
        #mean_pool=15  #Potentially add if expression for end size found in EEGNET class
        "learning_rate" : trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),
        "optimizer" : trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    }

    model = build_model(params)

    accuracy = train_and_eval(params, model)

    return accuracy



In [66]:
# Run Optuna
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=30)

[I 2022-11-01 14:17:26,792] A new study created in memory with name: no-name-d70f6691-bb02-4dad-85d4-c7cc14173bce
C:\Users\mathi\AppData\Local\Temp\ipykernel_12000\3789084798.py:44: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "dropout" : trial.suggest_uniform("dropout", 0.01, 1),
C:\Users\mathi\AppData\Local\Temp\ipykernel_12000\3789084798.py:48: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  "learning_rate" : trial.suggest_loguniform("learning_rate", 1e-6, 1e-1),


| epoch  1 | train loss 0.698228 | train acc 0.465587 | valid loss 0.705895 | valid acc 0.400000 |
| epoch  2 | train loss 0.704630 | train acc 0.461538 | valid loss 0.729865 | valid acc 0.400000 |
| epoch  3 | train loss 0.709771 | train acc 0.465587 | valid loss 0.744192 | valid acc 0.400000 |
| epoch  4 | train loss 0.711361 | train acc 0.457490 | valid loss 0.748013 | valid acc 0.400000 |
| epoch  5 | train loss 0.711595 | train acc 0.457490 | valid loss 0.748230 | valid acc 0.400000 |
| epoch  6 | train loss 0.711692 | train acc 0.461538 | valid loss 0.748343 | valid acc 0.400000 |
| epoch  7 | train loss 0.711726 | train acc 0.461538 | valid loss 0.748289 | valid acc 0.400000 |
| epoch  8 | train loss 0.711750 | train acc 0.461538 | valid loss 0.748520 | valid acc 0.400000 |
| epoch  9 | train loss 0.711764 | train acc 0.453441 | valid loss 0.748663 | valid acc 0.400000 |
| epoch 10 | train loss 0.711675 | train acc 0.457490 | valid loss 0.748549 | valid acc 0.400000 |
| epoch 11

[I 2022-11-01 14:19:20,791] Trial 0 finished with value: 0.4000000059604645 and parameters: {'filter_sizing': 3, 'dropout': 0.12674068874073185, 'Depth Parameter': 2, 'receptive_field': 256, 'learning_rate': 1.9116319511551443e-06, 'optimizer': 'SGD'}. Best is trial 0 with value: 0.4000000059604645.


| epoch 29 | train loss 0.711567 | train acc 0.457490 | valid loss 0.748764 | valid acc 0.400000 |
| epoch  1 | train loss 0.692819 | train acc 0.550607 | valid loss 0.698709 | valid acc 0.428571 |
| epoch  2 | train loss 0.692310 | train acc 0.554656 | valid loss 0.702504 | valid acc 0.400000 |
| epoch  3 | train loss 0.692024 | train acc 0.538462 | valid loss 0.705229 | valid acc 0.400000 |
| epoch  4 | train loss 0.691976 | train acc 0.542510 | valid loss 0.706037 | valid acc 0.400000 |
| epoch  5 | train loss 0.691927 | train acc 0.542510 | valid loss 0.706197 | valid acc 0.400000 |
| epoch  6 | train loss 0.691922 | train acc 0.546559 | valid loss 0.706289 | valid acc 0.400000 |
| epoch  7 | train loss 0.691861 | train acc 0.534413 | valid loss 0.706107 | valid acc 0.400000 |
| epoch  8 | train loss 0.691816 | train acc 0.534413 | valid loss 0.706074 | valid acc 0.400000 |
| epoch  9 | train loss 0.691761 | train acc 0.534413 | valid loss 0.706001 | valid acc 0.400000 |
| epoch 10

[I 2022-11-01 14:20:55,809] Trial 1 finished with value: 0.4000000059604645 and parameters: {'filter_sizing': 7, 'dropout': 0.5227548980879551, 'Depth Parameter': 1, 'receptive_field': 128, 'learning_rate': 1.0513522965650388e-05, 'optimizer': 'Adam'}. Best is trial 0 with value: 0.4000000059604645.


...

| epoch 29 | train loss 0.696980 | train acc 0.514170 | valid loss 0.681131 | valid acc 0.542857 |
| epoch  1 | train loss 0.701239 | train acc 0.538462 | valid loss 0.670817 | valid acc 0.542857 |
| epoch  2 | train loss 0.693735 | train acc 0.558704 | valid loss 0.691460 | valid acc 0.514286 |
| epoch  3 | train loss 0.660859 | train acc 0.570850 | valid loss 0.629768 | valid acc 0.685714 |
| epoch  4 | train loss 0.662307 | train acc 0.574899 | valid loss 0.619502 | valid acc 0.514286 |
| epoch  5 | train loss 0.655122 | train acc 0.591093 | valid loss 0.624404 | valid acc 0.657143 |
| epoch  6 | train loss 0.653286 | train acc 0.587045 | valid loss 0.640888 | valid acc 0.457143 |
| epoch  7 | train loss 0.644130 | train acc 0.611336 | valid loss 0.646011 | valid acc 0.685714 |
| epoch  8 | train loss 0.634872 | train acc 0.647773 | valid loss 0.621431 | valid acc 0.714286 |
| epoch  9 | train loss 0.622570 | train acc 0.619433 | valid loss 0.638067 | valid acc 0.542857 |
| epoch 10

[I 2022-11-01 15:38:54,856] Trial 28 finished with value: 0.6000000238418579 and parameters: {'filter_sizing': 2, 'dropout': 0.16397670257918293, 'Depth Parameter': 4, 'receptive_field': 448, 'learning_rate': 0.03466160385034222, 'optimizer': 'Adam'}. Best is trial 12 with value: 0.6857143044471741.


| epoch 29 | train loss 0.482465 | train acc 0.781377 | valid loss 0.972820 | valid acc 0.600000 |
| epoch  1 | train loss 0.693481 | train acc 0.502024 | valid loss 0.701127 | valid acc 0.428571 |
| epoch  2 | train loss 0.689833 | train acc 0.489879 | valid loss 0.710495 | valid acc 0.457143 |
| epoch  3 | train loss 0.683059 | train acc 0.542510 | valid loss 0.717569 | valid acc 0.514286 |
| epoch  4 | train loss 0.677858 | train acc 0.546559 | valid loss 0.724811 | valid acc 0.485714 |
| epoch  5 | train loss 0.672060 | train acc 0.546559 | valid loss 0.722665 | valid acc 0.457143 |
| epoch  6 | train loss 0.666532 | train acc 0.578947 | valid loss 0.725827 | valid acc 0.428571 |
| epoch  7 | train loss 0.658907 | train acc 0.615385 | valid loss 0.724158 | valid acc 0.485714 |
| epoch  8 | train loss 0.653535 | train acc 0.639676 | valid loss 0.723213 | valid acc 0.542857 |
| epoch  9 | train loss 0.649260 | train acc 0.643725 | valid loss 0.729494 | valid acc 0.542857 |
| epoch 10

[I 2022-11-01 15:41:49,793] Trial 29 finished with value: 0.5142857432365417 and parameters: {'filter_sizing': 2, 'dropout': 0.14898454338993072, 'Depth Parameter': 4, 'receptive_field': 384, 'learning_rate': 0.029314050639999808, 'optimizer': 'SGD'}. Best is trial 12 with value: 0.6857143044471741.


| epoch 29 | train loss 0.595912 | train acc 0.676113 | valid loss 0.690305 | valid acc 0.514286 |


In [73]:
import plotly
import sklearn
optuna.visualization.plot_param_importances(study)

ImportError: Tried to import 'plotly' but failed. Please make sure that the package is installed correctly to use this feature. Actual error: No module named 'plotly'.